In [1]:
import polychrom
import numpy as np 
import warnings
import h5py 
import glob
from polychrom.simulation import Simulation
import polychrom.starting_conformations
import polychrom.forces, polychrom.forcekits
import simtk.openmm 
import os 
import shutil
import polychrom.polymerutils

### Loading utils and forces from a reporter 


In [3]:
from polychrom.hdf5_format import hdf5Reporter, list_filenames, load_block, load_hdf5_file

### Making a simulation and passing a reporter 

In [4]:
%rm  test/*
data = polychrom.starting_conformations.grow_cubic(10000,30)

"""
Here we created a hdf5Reporter attached to a foler test, and we are saving 5 blocks per file 
(you should probalby use 50 here or 100. 5 is just for a showcase)
"""
reporter = hdf5Reporter(folder="test", max_data_length=5)


"""
Passing a reporter to the simulation object - many reporters are possible, and more will be added in a future
"""
sim = Simulation(N=10000, error_tol=0.001, collision_rate=0.01, integrator ="variableLangevin", platform="CPU", 
                reporters=[reporter])
sim.setData(data)
sim.addForce(polychrom.forcekits.polymerChains(sim))
sim._applyForces()
sim.addForce(polychrom.forces.sphericalConfinement(sim, density=0.1))
for i in range(19):
    sim.doBlock(10, saveExtras={"spam":[{},{}]})

"""
Here we are not forgetting to dump the last set of blocks that the reporter has. 
We have to do it at the end of every simulation. 

I tried adding it to the destructor to make it automatic,
but some weird interactions with garbage collection made it not very useable. 
"""
reporter.dump_data()


"Just doing a list of files at the end"
!ls -la test

rm: cannot remove 'test/*': No such file or directory


INFO:root:adding force HarmonicBonds 0
INFO:root:adding force Angle 1
INFO:root:adding force PolynomialRepulsive 2


Exclude neighbouring chain particles from PolynomialRepulsive
Number of exceptions: 9999


INFO:root:Particles loaded. Potential energy is 0.050224
INFO:root:block    1 pos[1]=[13.9 13.7 14.0] dr=0.21 t=0.9ps kin=8.39 pot=4.17 Rg=10.951 dt=23.8fs dx=15.38pm 
INFO:root:block    2 pos[1]=[13.9 13.6 13.9] dr=0.17 t=1.2ps kin=4.03 pot=9.08 Rg=10.951 dt=20.4fs dx=9.15pm 
INFO:root:block    3 pos[1]=[13.8 13.6 13.9] dr=0.09 t=1.4ps kin=7.83 pot=5.32 Rg=10.952 dt=24.2fs dx=15.10pm 
INFO:root:block    4 pos[1]=[13.8 13.6 13.9] dr=0.14 t=1.6ps kin=6.62 pot=6.53 Rg=10.956 dt=22.1fs dx=12.72pm 
INFO:root:block    5 pos[1]=[13.8 13.5 13.9] dr=0.11 t=1.8ps kin=6.87 pot=6.26 Rg=10.961 dt=21.8fs dx=12.78pm 
/home/magus/libs_tmp/polychrom/polychrom/hdf5_format.py:36: UserWarning: Could not convert record spam
  warnings.warn(f"Could not convert record {name}")
INFO:root:block    6 pos[1]=[13.9 13.5 13.9] dr=0.12 t=2.0ps kin=7.04 pot=6.07 Rg=10.965 dt=21.8fs dx=12.93pm 
INFO:root:block    7 pos[1]=[13.7 13.5 13.9] dr=0.12 t=2.2ps kin=7.06 pot=6.02 Rg=10.970 dt=21.8fs dx=12.96pm 


blocks_0-4.h5


...

blocks_5-9.h5


INFO:root:block   13 pos[1]=[13.6 13.9 13.8] dr=0.12 t=3.6ps kin=7.08 pot=5.86 Rg=11.012 dt=21.8fs dx=12.97pm 
INFO:root:block   14 pos[1]=[13.7 14.1 13.8] dr=0.13 t=3.8ps kin=7.27 pot=5.64 Rg=11.021 dt=21.8fs dx=13.15pm 
INFO:root:block   15 pos[1]=[13.8 14.2 13.8] dr=0.12 t=4.0ps kin=7.10 pot=5.79 Rg=11.031 dt=21.8fs dx=12.99pm 
INFO:root:block   16 pos[1]=[13.6 14.1 13.8] dr=0.13 t=4.2ps kin=7.22 pot=5.64 Rg=11.041 dt=21.8fs dx=13.10pm 
INFO:root:block   17 pos[1]=[13.5 14.1 13.7] dr=0.13 t=4.4ps kin=7.19 pot=5.65 Rg=11.052 dt=21.8fs dx=13.07pm 


blocks_10-14.h5


INFO:root:block   18 pos[1]=[13.5 14.3 13.6] dr=0.13 t=4.6ps kin=7.23 pot=5.59 Rg=11.064 dt=21.8fs dx=13.11pm 
INFO:root:block   19 pos[1]=[13.5 14.5 13.5] dr=0.13 t=4.9ps kin=7.12 pot=5.67 Rg=11.076 dt=21.8fs dx=13.01pm 


blocks_15-18.h5
total 6052
drwxrwxr-x 2 magus magus    4096 Aug 22 12:38 .
drwxrwxr-x 4 magus magus    4096 Aug 22 12:37 ..
-rw-rw-r-- 1 magus magus 1615688 Aug 22 12:38 applied_forces_0.hdf5
-rw-rw-r-- 1 magus magus 1150677 Aug 22 12:38 blocks_0-4.h5
-rw-rw-r-- 1 magus magus 1153017 Aug 22 12:38 blocks_10-14.h5
-rw-rw-r-- 1 magus magus  922721 Aug 22 12:38 blocks_15-18.h5
-rw-rw-r-- 1 magus magus 1152461 Aug 22 12:38 blocks_5-9.h5
-rw-rw-r-- 1 magus magus    6144 Aug 22 12:38 initArgs_0.hdf5
-rw-rw-r-- 1 magus magus  175219 Aug 22 12:38 starting_conformation_0.hdf5


In [5]:
files = list_filenames("test")
files   #  these are the paths for individual blocks

['test/blocks_0-4.h5::0',
 'test/blocks_0-4.h5::1',
 'test/blocks_0-4.h5::2',
 'test/blocks_0-4.h5::3',
 'test/blocks_0-4.h5::4',
 'test/blocks_5-9.h5::5',
 'test/blocks_5-9.h5::6',
 'test/blocks_5-9.h5::7',
 'test/blocks_5-9.h5::8',
 'test/blocks_5-9.h5::9',
 'test/blocks_10-14.h5::10',
 'test/blocks_10-14.h5::11',
 'test/blocks_10-14.h5::12',
 'test/blocks_10-14.h5::13',
 'test/blocks_10-14.h5::14',
 'test/blocks_15-18.h5::15',
 'test/blocks_15-18.h5::16',
 'test/blocks_15-18.h5::17',
 'test/blocks_15-18.h5::18']

In [6]:
"""
Loading the entire file, with position and other information
for that, use polychrom.hdf5_format.load_block

"""
load_block(files[0])

{'pos': array([[13.92293265, 13.71007105, 13.95973522],
        [12.9452834 , 13.89145657, 13.86133323],
        [13.11812076, 13.08613611, 14.03241599],
        ...,
        [14.13573444, 11.13731258, 14.23379963],
        [13.98083813, 12.01625943, 13.99015853],
        [14.02633324, 12.94941007, 13.90126466]]),
 'block': 1,
 'kineticEnergy': 8.390022275877323,
 'potentialEnergy': 4.1713218096788856,
 'time': 0.8870722680620542}

In [7]:
"""
It is backwards compatible with polymerutils.load as well, and it gives you just the XYZ
"""
polychrom.polymerutils.load(files[0])

array([[13.92293265, 13.71007105, 13.95973522],
       [12.9452834 , 13.89145657, 13.86133323],
       [13.11812076, 13.08613611, 14.03241599],
       ...,
       [14.13573444, 11.13731258, 14.23379963],
       [13.98083813, 12.01625943, 13.99015853],
       [14.02633324, 12.94941007, 13.90126466]])